In [17]:

import logging
import multiprocessing
import os
from collections import Counter
from typing import List, Union

from gensim.models import FastText
from indicnlp.tokenize.indic_tokenize import trivial_tokenize_indic
from tqdm import tqdm

tqdm.pandas()

In [2]:
# set up the logging to monitor gensim
logging.basicConfig(
    format="%(levelname)s - %(asctime)s: %(message)s",
    datefmt='%H:%M:%S',
    level=logging.INFO,
)

logger = logging.getLogger(__name__)

## Utilities

In [3]:
def tokenize_text(text: List[str]) -> List[List[str]]:
    """Tokenize text"""
    return [trivial_tokenize_indic(sent) for sent in tqdm(text, desc='tokenize', unit=' sentences')]

In [26]:
def train_fasttext(tokenized_text: List[List[str]], size: int = 100, window: int = 5, min_count: int = 1,
                   epochs: int = 10,
                   random_seed: int = 123, vec_file_path: Union[str, None] = None, ):
    """Learn fasttext embeddings"""
    # count the number of cores
    cores = multiprocessing.cpu_count()
    # create fasttext model
    model = FastText(
        size=size,
        window=window,
        min_count=min_count,
        workers=cores - 1,
        seed=random_seed,
    )
    # build vocab
    model.build_vocab(sentences=tokenized_text, progress_per=1000000)  # show progress after processing every 1M words
    # train
    model.train(sentences=tokenized_text, total_examples=model.corpus_count, epochs=epochs,
                report_delay=10)  # show progress after every 10 seconds
    if vec_file_path is not None:
        model.save(vec_file_path, binary=False)
    return model

## Load data
For learning the Odia word embeddings, we need monolingual Odia text data. You can possibly scrape data from an online source such as Wikipedia. For our experiments now, let's take the Odia monolingual text data available as part of the Indic NLP corpus.

In [5]:
filename = os.path.join('data/or')
assert os.path.isfile(filename)  # sanity check

In [6]:
with open(filename, 'r', encoding='utf-8') as f:
    lines = [s.strip() for s in tqdm(f.readlines(), desc='read lines from file')]

read lines from file: 100%|██████████| 3594672/3594672 [00:01<00:00, 1887993.40it/s]


In [7]:
# tokenize
tokens = tokenize_text(lines)

tokenize: 100%|██████████| 3594672/3594672 [01:18<00:00, 45640.83 sentences/s]


In [8]:
num_running_toks, num_unique_toks = 0, 0
counter = Counter()
for toks in tqdm(tokens, desc='compute frequencies of tokens', unit=' sentences'):
    counter.update(toks)

compute frequencies of tokens: 100%|██████████| 3594672/3594672 [00:20<00:00, 171988.99 sentences/s]


In [9]:
print(f'Number of sentences: {len(lines):,}')
print(f'Number of unique words or equivalantly, the size of vocabulary: {len(counter):,}')
print(f'Number of running words: {sum([freq for _, freq in counter.items()]):,}')

Number of sentences: 3,594,672
Number of unique words or equivalantly, the size of vocabulary: 778,862
Number of running words: 51,151,273


In [10]:

# most common words
counter.most_common(n=20)

[('।', 3393061),
 (',', 1191253),
 ('ଓ', 534792),
 ('ଏହି', 437185),
 ('ପାଇଁ', 373726),
 ('ସେ', 240775),
 ('ବୋଲି', 239837),
 ('ପରେ', 224959),
 ('କରି', 221628),
 ('ଏକ', 213516),
 ('ମଧ୍ୟ', 210907),
 ('ଏବଂ', 198988),
 ('କରିଥିଲେ', 195168),
 ('ସହ', 177040),
 ('-', 174796),
 ('ଖବର', 169373),
 ('.', 166728),
 ('କରିବା', 166276),
 ('ନେଇ', 161728),
 ('ବେଳେ', 156327)]

## Learn embeddings

In [11]:
fasttext_model = train_fasttext(tokenized_text=tokens, size=100, window=5, min_count=20, epochs=10, random_seed=123,
                                vec_file_path=os.path.join('embeddings.txt'))

INFO - 00:23:09: resetting layer weights
INFO - 00:23:19: collecting all words and their counts
INFO - 00:23:19: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 00:23:24: PROGRESS: at sentence #1000000, processed 14406915 words, keeping 356423 word types
INFO - 00:23:30: PROGRESS: at sentence #2000000, processed 28227997 words, keeping 518060 word types
INFO - 00:23:34: PROGRESS: at sentence #3000000, processed 42532970 words, keeping 692676 word types
INFO - 00:23:37: collected 778862 word types from a corpus of 51151273 raw words and 3594672 sentences
INFO - 00:23:37: Loading a fresh vocabulary
INFO - 00:23:37: effective_min_count=20 retains 72827 unique words (9% of original 778862, drops 706035)
INFO - 00:23:37: effective_min_count=20 leaves 49262024 word corpus (96% of original 51151273, drops 1889249)
INFO - 00:23:38: deleting the raw counts dictionary of 778862 items
INFO - 00:23:38: sample=0.001 downsamples 22 most-common words
INFO - 00:23:38: downsamp

In [29]:
# fasttext model's vocab size
print(f'fasttext model\'s vocabulary size: {len(fasttext_model.wv.vocab):,}')

fasttext model's vocabulary size: 72,827


## Evaluate embeddings
Here we evaluate the embeddings learned by just 👀 at the neighbours of a few words and examining if they are similar.



In [12]:
## find words similar to a given word
fasttext_model.wv.most_similar('ଗଛ', topn=10)  # TODO: misspell it

INFO - 01:07:19: precomputing L2-norms of word weight vectors
INFO - 01:07:19: precomputing L2-norms of ngram weight vectors


[('ଗଛଡାଳ', 0.8624626994132996),
 ('ବରଗଛ', 0.8271986842155457),
 ('ଗଛର', 0.8181042075157166),
 ('ଗଛଟି', 0.7930528521537781),
 ('ଆମ୍ବଗଛ', 0.7569365501403809),
 ('ଗଛକାଟି', 0.7496357560157776),
 ('ଫୁଲଗଛ', 0.7436925172805786),
 ('କଦଳୀଗଛ', 0.7428790926933289),
 ('ଗଛଟିଏ', 0.735636293888092),
 ('ଶାଳଗଛ', 0.7351911067962646)]

In [13]:
fasttext_model.wv.most_similar('ସଙ୍ଗୀତ', topn=10)

[('ସଂଗୀତ', 0.9560882449150085),
 ('ସଂଙ୍ଗୀତ', 0.9518374800682068),
 ('ସଙ୍ଗୀତର', 0.9251900315284729),
 ('ସଙ୍ଗୀତକାର', 0.8922179341316223),
 ('ସଂଗୀତର', 0.887881338596344),
 ('ସଂଗୀତକାର', 0.880925714969635),
 ('ସଙ୍ଗୀତଜ୍ଞ', 0.8672783374786377),
 ('ଗୀତିନାଟ୍ୟ', 0.8656123876571655),
 ('ନୃତ୍ୟଗୀତ', 0.8506015539169312),
 ('ନାଟ୍ୟ', 0.8301488757133484)]

In [14]:
fasttext_model.wv.most_similar('ଚଳଚ୍ଚିତ୍ର', topn=10)

[('ଚଳଚିତ୍ର', 0.9286938905715942),
 ('ଚଳଚ୍ଚିତ୍ରଟି', 0.9178325533866882),
 ('ଚଳଚ୍ଚିତ୍ରଟିର', 0.9093020558357239),
 ('ଚଳଚ୍ଚିତ୍ରର', 0.8912208080291748),
 ('ସିନେମାଟୋଗ୍ରାଫର', 0.8442597389221191),
 ('ସିନେମା', 0.835664689540863),
 ('ଚଳଚ୍ଚିତ୍ରରେ', 0.8248735666275024),
 ('ଫିଲ୍ମ', 0.8236875534057617),
 ('ସିନେ', 0.8232754468917847),
 ('ଚଳଚ୍ଚିତ୍ରଟିରେ', 0.8230992555618286)]

**Try some misspelled words**

In [15]:
fasttext_model.wv.most_similar('ଚଳଚ୍ଚିତ', topn=10)

[('ଚଳଚ୍ଚିତ୍ର', 0.8185211420059204),
 ('ଚଳଚ୍ଚିତ୍ରଟିର', 0.7849276065826416),
 ('ଚଳଚ୍ଚିତ୍ରଟି', 0.7687302231788635),
 ('ଚଳଚ୍ଚିତ୍ରର', 0.7509340047836304),
 ('ସମୁଚ୍ଚିତ', 0.7431687116622925),
 ('ସିନେମାଟୋଗ୍ରାଫର', 0.7184866666793823),
 ('ଲୋକପ୍ରିୟ', 0.717289388179779),
 ('ସିନେ', 0.7128420472145081),
 ('ଚଳଚ୍ଚିତ୍ରଟିରେ', 0.7066846489906311),
 ('ଫିଲ୍ମ', 0.7031621336936951)]

In [31]:
fasttext_model.wv.most_similar('ସଗୀତ', topn=10)

[('ଗୀତ', 0.9340874552726746),
 ('ନାଚଗୀତ', 0.892783522605896),
 ('ସଂଙ୍ଗୀତ', 0.8845723867416382),
 ('ନୃତ୍ୟଗୀତ', 0.8834997415542603),
 ('ସଂଗୀତ', 0.8760308027267456),
 ('ସଙ୍ଗୀତ', 0.87196946144104),
 ('ଗୀତ୍', 0.8635718822479248),
 ('ଗୀତିନାଟ୍ୟ', 0.8207296133041382),
 ('ଗୀତର', 0.8093352317810059),
 ('ନାଚଗୀତର', 0.806583821773529)]